# Applications of Gaussian Elimination

Suppose that a system of linear equations can be represented in the form $A \mathbf{x} = \mathbf{b}$ where 
$A = (a_{ij})$ is a $n \times n$ matrix and $\mathbf{x} = (x_1,\dots,x_n)^T$ and $\mathbf{b} = (b_1,\dots,b_n)^T$. 
Then we know from first year linear algebra that, if the system has a (unique) solution (which is equivalent to saying that $A$ is non-singular), then we find this solution by reducing the augmented matrix 
$$                                                                                                                 
    \begin{pmatrix}                                                                                               
      a_{11} & \cdots  & a_{1n} & b_1\\                                                                           
      \vdots  & \ddots & \vdots & \vdots \\                                                                       
      a_{n1}  & \cdots  & a_{nn} & b_n                                                                             
    \end{pmatrix}
$$                       
to a matrix of the form 
$$
   \begin{pmatrix}                                                                                               
      1 & \cdots  & 0 & c_1\\                                                                           
      \vdots  & \ddots & \vdots & \vdots \\                                                                       
      0  & \cdots  & 1 & c_n                                                                             
    \end{pmatrix}\, ,                                                                                             
$$
i.e. where the first $n$ columns are the $n \times n$ identity matrix, using elementary row operations. We then note that, relative to the original system  this represents 

\begin{align*}                                                                                                     
  x_1 + 0 + \cdots + 0 \;&=\; c_1 \\                                                                               
  0  + x_2 + \cdots + 0 \;&=\; c_2 \\                                                                             
                          &\hspace{2mm}\vdots \\ 
  0 + 0 + \cdots + x_n  \;&=\; c_n                                                                                 
\end{align*}  

so that the solution of the system of equations is given by the vector $\mathbf{c} = (c_1,\dots,c_n)^T$. In a similar vein, if $n \times n$ matrix $B = (b_{ij})$ is invertible (or equivalently if $B$ is non-singular) we can  find $B^{-1}$ as follows. We first form an $n \times 2n$ augmented matrix by juxtaposing $B$ with the $n \times n$ identity matrix $I$ - for which we use the notation  $[B \,|\, I]$- and perform elementary row operations until we obtain $[I\,|\,B^{-1}]$ and hence $B^{-1}$ itself.    

The purpose of the present question is to define both a function `my_solve` to  solve systems of linear equations, of the type described above, as also a function `my_inverse` to compute the inverse of square matrices. We shall do this in modular fashion by designing a function `reduce_matrix` that, given an $n \times m$ matrix $M$ such that $m \ge n$, performs elementary row operations on $M$ in such a way that (if successful) the first $n$ columns of $M$ end up forming the $n \times n$ identity matrix. We shall do this via parts (a), (b) and (c) below. 

**Notation.** To avoid confusing notation, for $n \times m$ matrix `M` we use python style notation to refer to its rows and columns and components. Accordingly `M[i,:]` denotes the row with index `i`  and `M[:,j]` denotes the column with index `j`. Thus for example `M[0,:]` and `M[n-1,:]` are respectively the first and last rows of `M`. Likewise `M[i,j]` denotes the component of `M` belonging to to row `M[i,:]` and column `M[:,j]`. So, as another example, `M[0,0]` is the first component  of the first row of `M` and `M[n-1,0]` is the first component of the last row of `M`.  For simplicity we will use the terminology *row* `i` and *column* `j` to refer to the row with index `i` and the column with index `j` (i.e. `M[i,:]` and `M[:,j]` respectively). 

This function `pivot` will take as input a $n \times m$ `numpy` matrix `M`, and an iteger `i` and accomplishes the following. 
1. If all the components in column  `i` below  and including `M[i,i]` are zero, it will print the message `"Error: pivot is zero"`  and return `i`. 
2. Otherwise we will ensure that `M[i,i]` has greatest absolute value relative to the components below it in  column `i`. To do this we will perform a swap of row `i` with an appropriate row below row `i`,  if necessary. Then divide row `i` by `M[i,i]` - so that after this operation we have `M[i,i]` = `1.0` - and return `-1` (which signals that the computation has been successful).      

**Example.** If `M` is the matrix
$
\begin{pmatrix}                                                                                                   
  -1 & 2 & -3 & 0 \\                                                                                               
  -3 & 4 & -1 & 2 \\                                                                                               
  5 & 2 & 0 & -2 \\                                                                                               
  3 & 7 & 2 & 5                                                                                                   
\end{pmatrix}
$
then `pivot(M,1)` will modify `M` so that it becomes 
$
\begin{pmatrix}                                                                                                   
  -1 & 2 & -3 & 0 \\                                                                                               
  3/7 & 1 & 2/7 & 5/7 \\                                                                                           
  5 & 2 & 0 & -2 \\                                                                                               
  -3 & 4 & -1 & 2                                                                                                 
\end{pmatrix}    
$
and will return `-1`. On the other hand if `N` is the matrix 
$
\begin{pmatrix}                                                                                                   
  -1 & 2 & -3 & 0 \\                                                                                               
  -3 & 0 & 4  & 1 \\                                                                                               
  2 & 0 & -1 & 5 \\                                                                                               
  1 & 0 & -2 & 3                                                                                                   
\end{pmatrix}  
$
then `pivot(N,1)` will not modify `N` but will print `"Error: pivot is zero"` and return  `1`. 

To test whether a component is zero we will use the function `is_zero` given below. Note we shall suppose that $n \le m$ and  `0` $\le$ `i` $<$ `n` .

In [11]:
import numpy as np
import numpy.linalg as lag

In [12]:
def is_zero(x): 
    return abs(x) < 1e-14

In [13]:
def pivot(M,i):
    em = "Error: pivot is zero"
    shape = np.shape(M)
    ec = True
    for n in range(i, shape[0]):
        if is_zero(M[n,i])==False:
            ec = False
    if ec == True:
        print(em)
        return i
    mv = 0
    p = 0
    for n in range(i,shape[0]):
        if abs(M[n,i])>mv:
            mv=abs(M[n,i])
            p = n    
    cm = M.copy()
    M[p,:] = cm[i,:]
    M[i,:] = cm[p,:]/cm[p,i]
    return(-1)

In [14]:
# tests:
# B = np.random.randint(-10,30,size=(5,6)) * 1.0 used to generate B
B = np.array([[ 8., 14.,  -7.,  0., -5.,  8.],
              [ 4.,  8.,  25., -2.,  6., 27.],
              [21.,  0.,   3., -2., -7., 16.],
              [13.,  3.,   6., 21.,  4., 23.],
              [20.,  0.,  -8., 12.,  4., -2.]])
C = np.array([[ 8., 14.,  -7.,  0., -5.,  8.],
              [ 4.,  8.,  25., -2.,  6., 27.],
              [21.,  0.,   0., -2., -7., 16.],
              [13.,  3.,   0., 21.,  4., 23.],
              [20.,  0.,   0., 12.,  4., -2.]])
# Now the pivoting
B_val = pivot(B,2)
C_val = pivot(C,2)
# B should have been modified and should be identical to B_result below
B_result = np.array([[ 8.  , 14.  , -7.  ,  0.  , -5.  ,  8.  ],
                     [ 4.  ,  8.  , 25.  , -2.  ,  6.  , 27.  ],
                     [-2.5 ,  0.  ,  1.  , -1.5 , -0.5 ,  0.25],
                     [13.  ,  3.  ,  6.  , 21.  ,  4.  , 23.  ],
                     [21.  ,  0.  ,  3.  , -2.  , -7.  , 16.  ]])
# C should not have been modified. C should be identical to C_result below. 
C_result = np.array([[ 8., 14., -7.,  0., -5.,  8.],
                    [ 4.,  8., 25., -2.,  6., 27.],
                    [21.,  0.,  0., -2., -7., 16.],
                    [13.,  3.,  0., 21.,  4., 23.],
                    [20.,  0.,  0., 12.,  4., -2.]]) 
assert B_val == -1
assert np.allclose(B,B_result)
assert C_val == 2
assert np.allclose(C,C_result)
# Note pivot(C,2) generates message "Error: pivot is zero" below. This is correct behaviour

Error: pivot is zero


Now we will write a function `gauss` that takes as input a $n \times m$ numpy matrix `M` and an integer `i` - where it is assumed that `M[i,i]` = `1` - and peforms elementary row operations on the rows of `M` in such a way that, following this operation, all the components of column `i` except  `M[i,i]` are zero. The function should return `-1`.  

**Example.** Suppose that `M` is the matrix 
$
\begin{pmatrix}                                                                                                   
  -1 & 2 & -3 & 0 \\                                                                                               
  3/7 & 1 & 2/7 & 5/7 \\                                                                                           
  5 & 2 & 0 & -2 \\                                                                                               
  -3 & 4 & -1 & 2                                                                                                 
\end{pmatrix}
$. 
Then `gauss(M,1)` will modify `M` so that it becomes 

$$
\begin{pmatrix}                                                                                                   
  -1 - 6/7 & 2-2 & -3-4/7 & 0-10/7 \\
  3/7 & 1 & 2/7 & 5/7 \\                                                                                           
  5- 6/7 & 2-2 & 0-4/7 & -2-10/7 \\
  -3-12/7 & 4-4 & -1-8/7 & 2-20/7                                                                                 
\end{pmatrix}
\;=\;
\begin{pmatrix}                                                                                                   
  -13/7  & 0 & -25/7 & -10/7 \\
  3/7 & 1 & 2/7 & 5/7 \\                                                                                           
  29/7 & 0 & -4/7 & -24/7 \\
  -33/7 & 0 & -15/7 & -6/7                                                                                 
\end{pmatrix} \,.
$$

**Note.** We will only implement `gauss(M,i)` on a matrix that has already been successfully processed by our function `pivot(M,i)`. This is why we can assume that `M[i,i]` = `1`. Accordingly there is no need to test for incompatible input.

In [15]:
def gauss(M,i):
    cm = M.copy()
    shape = np.shape(M)
    for n in range(shape[0]):
        M[n,:] = cm[n,:] - (M[n,i] * cm[i,:])
    M[i,:] = cm[i,:]
    return(-1)

In [16]:
# tests
B1 = np.array([[ 8.  , 14.  , -7.  ,  0.  , -5.  ,  8.  ],
               [ 4.  ,  8.  , 25.  , -2.  ,  6.  , 27.  ],
               [-2.5 , -0.  ,  1.  , -1.5 , -0.5 ,  0.25],
               [13.  ,  3.  ,  6.  , 21.  ,  4.  , 23.  ],
               [21.  ,  0.  ,  3.  , -2.  , -7.  , 16.  ]])
B1_val = gauss(B1,2)
# B1 has now been modified (via elementary row operations)
# B1 should now be the same as B1_result below
B1_result = np.array([[ -9.5 ,  14.  ,   0.  , -10.5 ,  -8.5 ,   9.75],
                      [ 66.5 ,   8.  ,   0.  ,  35.5 ,  18.5 ,  20.75],
                      [ -2.5 ,  -0.  ,   1.  ,  -1.5 ,  -0.5 ,   0.25],
                      [ 28.  ,   3.  ,   0.  ,  30.  ,   7.  ,  21.5 ],
                      [ 28.5 ,   0.  ,   0.  ,   2.5 ,  -5.5 ,  15.25]])
assert B1_val == -1
assert np.allclose(B1,B1_result)

Notice that, given a $n \times m$ `numpy` matrix `M`, with $m \ge n$,  if  we start by computing `pivot(M,0)` and this returns `-1` we can proceed by  computing `gauss(M,0)` and so reduce the first column of `M` to an array with first component `1` and all other components `0`. Then continuing,  if we next compute `pivot(M,1)` and this returns `-1` then we can proceed by  computing `gauss(M,1)` and so reduce the second column of `M` to an array with second component `1` and all other components `0`. Continuing in this way - provided `pivot(M,i)` does indeed return `-1` at every step `i` = `0`,$\dots$,`n-1` - after $n$ steps we will have reduced `M` so that its first $n$ columns are the $n \times n$ identity matrix `I`. 

Using this idea, we will write a function `matrix_reduce` that takes as input a $n \times m$ numpy matrix `M`, and tries to row reduce `M` to a matrix whose first $n$ columns are the $n \times n$ identity matrix. The function will use the functions `pivot` and `gauss`. If at some stage `i` the computation `pivot(M,i)` returns `i`  the function should immediately return `i` (so effectively aborting the computation). If, on the other hand this does not happen at any stage `i` = `0`,$\dots$,`n-1` (meaning that the reduction of `M` was successful) the function should return `-1`. 

We will assume that $m \ge n$.

In [17]:
def reduce_matrix(M):
    shape = np.shape(M)
    for i in range(0,shape[0]):
        if pivot(M,i) == -1:
            gauss(M,i)
        else:
            return(i)
    return(-1)

In [18]:
# tests:
B2 = np.array([[ 8., 14.,  -7.,  0., -5.,  8.,  3.],
               [ 4.,  8.,  25., -2.,  6., 27., -1.],
               [21.,  0.,   3., -2., -7., 16., 11.],
               [13.,  3.,   6., 21.,  4., 23., -5.],
               [20.,  0.,  -8., 12.,  4., -2., 12.]])
B2_val = reduce_matrix(B2)
# B2 has now been reduced and should be the same as B2_result below
B2_result = np.array([[ 1.,  0.,  0.,  0.,  0., 0.20081312,  0.7470541 ],
                      [ 0.,  1.,  0.,  0.,  0., 0.59194741, -0.12856537],
                      [ 0.,  0.,  1.,  0.,  0., 1.25116973, -0.35013236],
                      [ 0.,  0.,  0.,  1.,  0., 0.79038592, -0.72034135],
                      [ 0.,  0.,  0.,  0.,  1., -1.37288388, 0.72548882]])
assert B2_val == -1
assert np.allclose(B2,B2_result)
# Column C2[:,3]  is a scalar multiple of column C2[:,1] in C2 
C2 = np.array([[ 8., 14.,  -7.,  7., -5.,  8.,  3.],
               [ 4.,  8.,  25.,  4.,  6., 27., -1.],
               [21.,  0.,   3.,  0., -7., 16., 11.],
               [13.,  3.,   6., 1.5,  4., 23., -5.],
               [20.,  0.,  -8.,  0.,  4., -2., 12.]])
C2_val = reduce_matrix(C2)
# This time the computation is aborted as pivot(C2,3) fails
assert C2_val == 3 
# Note that reduce_matrix(C2) should print the message "Error: pivot is zero" below. 

Error: pivot is zero


Suppose now that we are given a system of $n$ linear equations in $n$ unknowns so that we are able to represent the system in the form  $A\mathbf{x} = \mathbf{b}$ where $A$ is a $n \times n$ matrix. Then, provided the system has a (unique) solution, we can use the function `matrix_reduce` to find the solution. We will write just such a function `my_solve` which takes as input a $n \times n$ numpy matrix `A` and a column vector `b` in the form of a $n \times 1$ numpy array. This function will form the augmented $n \times (n+1)$ matrix `M` with `A` comprising the first $n$ columns
of `M` and `b` making up the last column. The function will then pass `M` as input to the function `reduce_matrix`. If `reduce_matrix` succeeds in reducing `M` (i.e. in the manner  described in part (c)) then the function will return the last column of `M` as a $3 \times 1$ numpy array. If `reduce_matrix` does not succeed the function should simply pass on/return integer `i` returned by `reduce_matrix`.  

**Note.** The matrix `A` is singular if and only if the linear system has either no solution or infinitely many solutions. In this case, the function `pivot`, working as a subfunction of `reduce_matrix`, will encounter a zero at some stage `i` in the computation - i.e. when `pivot(M,i)` is being processed -  and so the error message `i` will be passed/returned from `pivot` to `reduce_matrix` to `my_solve` signalling that the computation has been aborted. If on the other hand `A` is non-singular then `my_solve` will return the solution of the system in the form of a $3 \times 1$ `numpy` array.

In [19]:
def my_solve(A,b):
    shape = np.shape(A)[0]
    M = np.empty((shape, shape + 1))
    M[0:shape, 0:shape] = A
    M[0:shape,shape] = list(b)
    result = reduce_matrix(M)
    if result == -1:
        ans = np.empty((shape,1))
        for z in range(shape):
            ans[z,0] = M[z,shape]
        return(ans)
    else:
        return(result)

In [20]:
A3 = np.array([[1.0, 2, -3], [2, -1, 4], [1, -1, 1]])
b3 = np.array([[9.0],[1],[0]])
x3 = my_solve(A3,b3)
# x3 should now be the solution of this system. 
print("x3 = \n", x3, "\n")
assert np.allclose(x3,lag.solve(A3,b3))
# There is not a unique solution for the following system
B3 = np.array([[1.0, 2, -3], [2, -1, 4], [3.0, 6, -9]])
b3 = np.array([[9.0],[1],[0]])
y3 = my_solve(B3,b3)
# In fact pivot([B3|b3],2) failed during the computation
assert y3 == 2
# The message "Error: pivot is zero" should appear below 

x3 = 
 [[ 2.8]
 [ 2.2]
 [-0.6]] 

Error: pivot is zero


Now we will write a function `my_inverse` which takes as input a $n \times n$ matrix `M`, constructs the $n \times 2n$ augmented matrix `N`, whose first $n$ columns comprise `M` and whose last $n$ columns comprise  the $n \times n$ identity matrix `I`, and then passes `N` to `matrix_reduce`. If the reduction performed by `matrix_reduce` aborts/fails this function will return the integer `i` passed to it by `matrix_reduce`.  If the reduction is successful then the function will return the inverse of `M`.

In [21]:
def my_inverse(M):
    shape = np.shape(M)[0]
    N = np.empty((shape, 2*shape))
    N[0:shape,0:shape] = M
    N[0:shape,shape:2*shape] = np.identity(shape)
    ans = reduce_matrix(M)
    if ans != -1:
        return(ans)
    else:
        inv = N[0:shape,shape:2*shape]
        print(inv)
        return(inv)

In [22]:
#A4 = np.random.randint(-10,30,size=(4,4)) * 1.0
A4 = np.array([[12.,  5., 24., 29.],
               [ 1., 16.,  9., -2.],
               [ 8.,  9., -5., -4.],
               [ 0., -5.,  4.,  3.]])
A4_inv = my_inverse(A4)
assert np.allclose(np.dot(A4,A4_inv), np.eye(4))
# B4 is not invertible 
B4 = np.array([[12.,  5., 24., -6.],
               [ 1., 16.,  9., -.5],
               [ 8.,  9., -5., -4.],
               [ 0., -5.,  4.,  0.]])
B4_inv = my_inverse(B4)
# In fact pivot([B4|I],3) failed during the computation
assert B4_inv == 3
assert my_inverse(np.zeros((7,7))) == 0 
# The message  below "Error: pivot is zero" should appear twice below. 

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
Error: pivot is zero
Error: pivot is zero
